## LFR benchmark for Altmap vs Map Eq
### Compare altmap to map eq using networkx

In [3]:
from matplotlib.pyplot import Line2D
from matplotlib.colors import ListedColormap

from altmap.altmap_helpers.lfr import *

# show plots in separate window
%pylab

# init rc params
init_plt_params()

# define colormap
colors = np.array([[255,   0,   0, 255],
                   [  0, 255,   0, 255],
                   [100,   0, 100, 255],
                   [  0,   0, 255, 255],
                   [255, 255,   0, 255],
                   [  0, 255, 255, 255],
                   [200,   0, 100, 255],
                   [  0,   0, 100, 255],
                   [240, 130,   0, 255],
                   [255,   0, 255, 255],
                   [  0, 100,   0, 255],
                   [100,   0,   0, 255],
                   [  0,   0,   0, 255]]) / 255.0
commColors = ListedColormap(colors, name='commColors', N=len(colors))

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [114]:
G, communities_true, num_communities_true = generate_LFR_benchmark(N=600, mu=0.35)
_, _, comm_sizes = nodes_per_community(communities_true)
print (f'There are {num_communities_true} ground truth communities.')
print (f'Community sizes are {comm_sizes}')

communities_infomap, _, _, _ = infomap(G, altmap=False)
communities_altmap, _, _, _ = infomap(G, altmap=True, update_inputfile=False)
communities_altmap_sci, _, communities_sci, _ = infomap(G, altmap=True, init='sc', update_inputfile=False)
#             
# print(f'Altmap cost for true partition is {altmap_cost(G, communities_true)}')

There are 8 ground truth communities.
Community sizes are [ 79  64 106  60  42  81 118  50]
Spectral clustering finished in 0.6184749999999894 seconds.


/home/chri/python_environments/myDevEnv/lib/python3.7/site-packages/networkx/readwrite/pajek.py:77: UserWarning: Node attribute community is not processed. Non-string attribute.
  'Non-string attribute'))


In [116]:
def relabel_communities_by_largest(communities):
    comm_ids, comm_nodes, comm_sizes = nodes_per_community(communities)
    tmp = sorted(zip(comm_sizes, comm_ids, comm_nodes), reverse = True)
    sorted_nodes = [node for _,_,nodes in tmp for node in nodes]
    sorted_comm_sizes = [comm_size for comm_size,_,_ in tmp]
    sorted_comms = [i for i, comm_size in enumerate(sorted_comm_sizes) for j in range(comm_size)]
    relabeled_communities = OrderedDict(zip(sorted_nodes, sorted_comms))
    return sorted_nodes, sorted_comms, relabeled_communities

def plot_communities(communities, pos, num_communities_true, aggregate_comms=True):
    # plotting params
    node_size = 250
    edge_width = 0.2
    font_size = 13
    cmap = plt.get_cmap('Set3') # 'Set3' 'jet' 'hsv' commColors
    
    # plot communities
    fig, ax = plt.subplots(figsize=(14,9))
    ax.set_frame_on(False)
    ax.set_yticklabels([])
    ax.set_yticks([])
    ax.set_xticklabels([])
    ax.set_xticks([])
    # ax.set_title('Ground truth')
    # nx.draw_networkx_edges(G, pos, ax=ax, width=edge_width)
    
    num_colors_used = min(num_communities_true + 1, get_num_communities(communities))
    _,_, relabeled_comms = relabel_communities_by_largest(communities)
    comm_ids, comm_nodes, comm_sizes = nodes_per_community(relabeled_comms)
    tmp = sorted(zip(comm_sizes, comm_ids, comm_nodes), reverse = True)
    # print each community seperately with its own color
    for _, comm_id, nodes in tmp:
        c_idx = comm_id if comm_id < num_colors_used or not aggregate_comms else num_colors_used - 1
        nx.draw_networkx_nodes(G, pos=pos, node_color=[cmap(c_idx)], cmap=cmap, ax=ax, node_size=node_size, 
                               nodelist=nodes)
    # nx.draw_networkx_labels(G, pos, ax=ax, labels=communities, font_weight='bold', font_size=font_size)
    
    comm_sizes_aggregated = comm_sizes
    if aggregate_comms:
        comm_sizes_aggregated = sorted(comm_sizes, reverse=True)
        comm_sizes_aggregated[num_colors_used-1] = sum(comm_sizes_aggregated[num_colors_used-1:])
        comm_sizes_aggregated = comm_sizes_aggregated[:num_colors_used]

    # assemble custom legend for community colors
    custom_lines = [Line2D([], [], color=cmap(c), marker='o', ms=15, lw=0) for c in range(num_colors_used)]
    custom_labels = [f'Community {id} ({size} nodes)' for id, size in enumerate(comm_sizes_aggregated)]
    
    if aggregate_comms:
        custom_labels[-1] = f'Residual nodes ({comm_sizes_aggregated[-1]})'
    
    if num_colors_used >= 4:
        res = num_colors_used % 4
        split_size = int(num_colors_used / 4)
        
        start = 0; end = split_size + int(res > 0)
        slc = slice(start, end)
        leg3 = ax.legend(custom_lines[slc], custom_labels[slc], loc=2, fontsize='small')
        
        start += split_size + int(res > 0); end += split_size + int(res > 1)
        slc = slice(start, end)
        leg2 = ax.legend(custom_lines[slc], custom_labels[slc], loc=1, fontsize='small')
        
        start += split_size + int(res > 1); end += split_size + int(res > 2)
        slc = slice(start, end)
        leg1 = ax.legend(custom_lines[slc], custom_labels[slc], loc=3, fontsize='small')
        
        start += split_size + int(res > 2);
        slc = slice(start, None)
        ax.legend(custom_lines[slc], custom_labels[slc], loc=4, fontsize='small')
        ax.add_artist(leg1)
        ax.add_artist(leg2)
        ax.add_artist(leg3)
    else:
        ax.legend(custom_lines, custom_labels, loc=2, fontsize='small')
    
    plt.tight_layout()
    return fig
        

plt.close('all')
pos = community_layout(communities_true)
fig1 = plot_communities(communities_true, pos, num_communities_true, aggregate_comms=False)
fig2 = plot_communities(communities_infomap, pos, num_communities_true, aggregate_comms=False)
fig3 = plot_communities(communities_altmap, pos, num_communities_true)
fig4 = plot_communities(communities_altmap_sci, pos, num_communities_true)

In [55]:
dpi = 400
fig_path = '/home/chri/lfr_sample_true.png'
fig1.savefig(fig_path, dpi=dpi, format='png')

fig_path = '/home/chri/lfr_sample_infomap.png'
fig2.savefig(fig_path, dpi=dpi, format='png')

fig_path = '/home/chri/lfr_sample_altmap.png'
fig3.savefig(fig_path, dpi=dpi, format='png')

fig_path = '/home/chri/lfr_sample_altmap_sci.png'
fig4.savefig(fig_path, dpi=dpi, format='png')

In [89]:
dpi = 600
fig_path = '/home/chri/lfr_sample_true.pdf'
fig1.savefig(fig_path, dpi=dpi, format='pdf')

fig_path = '/home/chri/lfr_sample_infomap.pdf'
fig2.savefig(fig_path, dpi=dpi, format='pdf')

fig_path = '/home/chri/lfr_sample_altmap.pdf'
fig3.savefig(fig_path, dpi=dpi, format='pdf')

fig_path = '/home/chri/lfr_sample_altmap_sci.pdf'
fig4.savefig(fig_path, dpi=dpi, format='pdf')